In [ ]:
import taichi as ti
import taichi_glsl as ts
import taichi_three as t3

In [ ]:
import dataclasses as dc
import os

import torch
import numpy as np
from PIL import Image

from pydrake.math import RollPitchYaw, RotationMatrix, RigidTransform
from pydrake.common.eigen_geometry import Quaternion

import neural_renderer as nr
from simple_mesh import Mesh, intrinsic_matrix_from_fov, RgbDepth

In [ ]:
full_state0 = np.load("full_state0.npy", allow_pickle=True).item()
full_state0_img = Image.open("full_state0.png")
x, _v, _F, _C, primitive_states = full_state0["state"]
cyl_p = primitive_states[:3]
cyl_qwxyz = primitive_states[3:]
cyl_R = Quaternion(cyl_qwxyz).rotation().copy()
display(full_state0_img)

In [ ]:
ti.init(ti.gpu)

In [ ]:
pos = ti.Vector.field(3, ti.f32, len(x))
pos.from_numpy(x)

In [ ]:
scene = t3.Scene()

particles = t3.ScatterModel(radius=4)
particles.particles = pos
scene.add_model(particles)

obj = t3.readobj("textured_cylinder.obj")
obj["vp"] = obj["vp"][:, :3]  # shrug
r = 0.0254
h = 0.254
obj["vp"] *= np.array([[r, h, r]])
texture = ti.imread("textured_cylinder.png")
cylinder = t3.Model.from_obj(obj, texture)
scene.add_model(cylinder)

camera = t3.Camera(
#     pos=np.array([0.5, 2.5, 2.0]).tolist(),
    pos=[0.5, 0.8 , 1.5],
    target=x.mean(axis=0).tolist(),
)
scene.add_camera(camera)

light = t3.Light()
scene.add_light(light)

In [ ]:
cylinder.L2W.offset.from_numpy(cyl_p)
cylinder.L2W.matrix.from_numpy(cyl_R)

In [ ]:
cyl_R

In [ ]:
camera.pos.to_numpy()

In [ ]:
cylinder.L2W.offset.to_numpy()

In [ ]:
# pos.from_numpy(np.zeros_like(x))

In [ ]:
gui = ti.GUI("test", camera.res)
while gui.running:
    gui.get_event(None)
    gui.running = not gui.is_pressed(ti.GUI.ESCAPE)
    camera.from_mouse(gui)
    scene.render()
    gui.set_image(camera.img)
    gui.show()